# Lab02: Working with Time Series


## 1. Thu thập dữ liệu

- Ngữ cảnh để nhóm chúng em tìm kiếm dữ liệu này:
- Chủ đề dữ liệu:
- Nguồn dữ liệu:
- License của dữ liệu:
- Phương pháp để thu thập dữ liệu này:


## 2. Tiền xử lý dữ liệu

### 2.1. Mỗi dòng có ý nghĩa gì? ? Có vấn đề các dòng có ý nghĩa khác nhau không?

### 2.2. Mỗi cột có ý nghĩa gì?


### 2.3. Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

### 2.4. Với mỗi cột, các giá trị (dạng số, dạng phân loại) được phân bố như thế nào?

### 2.5. Có cần phải tiền xử lý dữ liệu hay không và nếu có thì nhóm sinh viên cần phải xử lý như thế nào?

## 3. Khám phá mối quan hệ trong dữ liệu

Chúng ta sẽ khám phá các mối quan hệ trong dữ liệu bằng cách đưa ra những câu hỏi và trả lời.

### 3.1. Đặt câu hỏi

**1. Câu hỏi 1:**

**2. Câu hỏi 2:**

**3. Câu hỏi 3:**

**4. Câu hỏi 4:**

**5. Câu hỏi 5:**

**6. Câu hỏi 6:**

**7. Câu hỏi 7:**

**8. Câu hỏi 8:**


### 3.2. Trả lời câu hỏi

#### **Câu hỏi 1:** lorem ipsum dolor sir almet...

#### **Câu hỏi 2:** lorem ipsum dolor sir almet...

#### **Câu hỏi 3:** lorem ipsum dolor sir almet...